In [1]:
import itertools
import networkx as nx
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import sys
sys.path.append('/Users/ziniuwu/Desktop/research/BayesCard')
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import (
    WeightedMinFill,
    MinNeighbors,
    MinFill,
    MinWeight,
)
from Pgmpy.factors.discrete import TabularCPD

In [2]:
import numpy as np
import pandas as pd
import time
import sys
import copy
from Models.pgmpy_BN import Pgmpy_BN
from Testing.toy_dataset import *

In [28]:
df = toy_data_slightly_correlated_cat(nrows=100000, return_df=True)
df = df.drop("id", axis=1)
df.head()

,cat_attr1,cat_attr2,cat_attr3,cat_attr4,cat_attr5,cat_attr6,cat_attr7,cat_attr8
0,5,2,8,5,4,9,2,7
1,0,8,8,0,18,18,8,56
2,3,7,10,3,6,9,8,19
3,3,0,4,3,7,10,0,34
4,7,4,11,7,29,36,4,4


In [29]:
BN = Pgmpy_BN('dmv')
BN.build_from_data(df, algorithm="exact", max_parents=3, ignore_cols=['id'], sample_size=100000)

Discretizing table takes 0.5326049327850342 secs
Structure learning took 6.831698894500732 secs.
done, parameter learning took 0.09351301193237305 secs.


In [30]:
import itertools
import networkx as nx
import numpy as np
import time
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import copy
from Pgmpy.inference import Inference
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import (
    WeightedMinFill,
    MinNeighbors,
    MinFill,
    MinWeight,
)
from Pgmpy.factors.discrete import TabularCPD

class VariableElimination(object):
    def __init__(self, model, probs=None, root=True):
        model.check_model()
        self.model = model
        if probs is not None:
            self.probs = probs
        else:
            self.probs = dict()

        if isinstance(model, JunctionTree):
            self.variables = set(chain(*model.nodes()))
        else:
            self.variables = model.nodes()

        self.cardinality = {}
        self.factors = defaultdict(list)

        if isinstance(model, BayesianModel):
            self.state_names_map = {}
            for node in model.nodes():
                cpd = model.get_cpds(node)
                if isinstance(cpd, TabularCPD):
                    self.cardinality[node] = cpd.variable_card
                    cpd = cpd.to_factor()
                for var in cpd.scope():
                    self.factors[var].append(cpd)
                self.state_names_map.update(cpd.no_to_name)

        elif isinstance(model, JunctionTree):
            self.cardinality = model.get_cardinality()

            for factor in model.get_factors():
                for var in factor.variables:
                    self.factors[var].append(factor)
        if root:
            self.root = self.get_root()

    def get_root(self):
        """Returns the network's root node."""

        def find_root(graph, node):
            predecessor = next(self.model.predecessors(node), None)
            if predecessor:
                root = find_root(graph, predecessor)
            else:
                root = node
            return root

        return find_root(self, list(self.model.nodes)[0])

    def steiner_tree(self, nodes):
        """Returns the minimal part of the tree that contains a set of nodes."""
        sub_nodes = set()

        def walk(node, path):
            if len(nodes) == 0:
                return

            if node in nodes:
                sub_nodes.update(path + [node])
                nodes.remove(node)

            for child in self.model.successors(node):
                walk(child, path + [node])

        walk(self.root, [])
        sub_graph = self.model.subgraph(sub_nodes)
        sub_graph.cardinalities = defaultdict(int)
        for node in sub_graph.nodes:
            sub_graph.cardinalities[node] = self.model.cardinalities[node]
        return sub_graph

    def get_probs(self, attribute, values):
        """
        Calculate Pr(attr in values) where values must be a list
        """
        factor = self.probs[attribute]
        values = [factor.get_state_no(attribute, no) for no in values]
        return np.sum(factor.values[values])


    def _get_working_factors(self, variables=[], evidence=None, return_probs=False, reduce=True, BP=False):
        """
        Uses the evidence given to the query methods to modify the factors before running
        the variable elimination algorithm.
        Parameters
        ----------
        evidence: dict
            Dict of the form {variable: state}
        Returns
        -------
        dict: Modified working factors.
        """
        if BP:
            working_factors = {
                node: {(factor, None) for factor in self.factors[node]}
                for node in self.factors
            }
            print(working_factors)
            # Dealing with evidence. Reducing factors over it before VE is run.
            if evidence:
                for evidence_var in evidence:
                    for factor, origin in working_factors[evidence_var]:
                        factor_reduced = factor.reduce(
                            [(evidence_var, evidence[evidence_var])], inplace=False
                        )
                        for var in factor_reduced.scope():
                            if var in working_factors:
                                working_factors[var].remove((factor, origin))
                                working_factors[var].add((factor_reduced, evidence_var))
                    if type(evidence[evidence_var]) != list:
                        del working_factors[evidence_var]
            print(working_factors)
            return working_factors

        else:
            useful_var = copy.deepcopy(variables)
            if evidence:
                useful_var += list(evidence.keys())
            sub_graph_model = self.steiner_tree(useful_var)
            variables_sub_graph = set(sub_graph_model.nodes)

            working_factors = dict()
            for node in sub_graph_model.nodes:
                working_factors[node] = set()
                for factor in self.factors[node]:
                    if set(factor.variables).issubset(variables_sub_graph):
                        working_factors[node].add((factor, None))

            if return_probs:
                probs = dict()
            # Dealing with evidence. Reducing factors over it before VE is run.
            if evidence and reduce:
                for evidence_var in evidence:
                    for factor, origin in working_factors[evidence_var]:
                        factor_reduced = factor.reduce(
                            [(evidence_var, evidence[evidence_var])], inplace=False
                        )
                        if return_probs:
                            factor_reduced.normalize()
                            probs[evidence_var] = self.get_probs(evidence_var, evidence[evidence_var])
                        for var in factor_reduced.scope():
                            if var in working_factors:
                                working_factors[var].remove((factor, origin))
                                working_factors[var].add((factor_reduced, evidence_var))
                    if type(evidence[evidence_var]) != list:
                        del working_factors[evidence_var]
            if return_probs:
                return working_factors, sub_graph_model, probs
            return working_factors, sub_graph_model

    def _get_elimination_order(
            self, variables=None, evidence=None, model=None, elimination_order="minfill", show_progress=False
    ):
        """
        Deals with all elimination order parameters given to _variable_elimination method
        and returns a list of variables that are to be eliminated
        Parameters
        ----------
        elimination_order: str or list
        Returns
        -------
        list: A list of variables names in the order they need to be eliminated.
        """
        if model is None:
            model = self.model
        if isinstance(model, JunctionTree):
            all_variables = set(chain(*model.nodes()))
        else:
            all_variables = model.nodes()

        if variables is None:
            to_eliminate = set(all_variables)
        else:
            not_evidence_eliminate = []
            if evidence is not None:
                for key in evidence:
                    if type(evidence[key]) != list:
                        not_evidence_eliminate.append(key)
            to_eliminate = (
                    set(all_variables)
                    - set(variables)
                    - set(not_evidence_eliminate)
            )

        # Step 1: If elimination_order is a list, verify it's correct and return.
        if hasattr(elimination_order, "__iter__") and (
                not isinstance(elimination_order, str)
        ):
            if any(
                    var in elimination_order
                    for var in set(variables).union(
                        set(evidence.keys() if evidence else [])
                    )
            ):
                raise ValueError(
                    "Elimination order contains variables which are in"
                    " variables or evidence args"
                )
            else:
                return elimination_order

        # Step 2: If elimination order is None or a Markov model, return a random order.
        elif (elimination_order is None) or (not isinstance(model, BayesianModel)):
            return to_eliminate

        # Step 3: If elimination order is a str, compute the order using the specified heuristic.
        elif isinstance(elimination_order, str) and isinstance(
                model, BayesianModel
        ):
            heuristic_dict = {
                "weightedminfill": WeightedMinFill,
                "minneighbors": MinNeighbors,
                "minweight": MinWeight,
                "minfill": MinFill,
            }
            elimination_order = heuristic_dict[elimination_order.lower()](
                model
            ).get_elimination_order(nodes=to_eliminate, show_progress=show_progress)
            return elimination_order

    def _variable_elimination(
            self,
            variables,
            operation,
            evidence=None,
            elimination_order="minfill",
            joint=True,
            show_progress=False,
            BP=False
    ):
        """
        Implementation of a generalized variable elimination.

        Parameters
        ----------
        variables: list, array-like
            variables that are not to be eliminated.

        operation: str ('marginalize' | 'maximize')
            The operation to do for eliminating the variable.

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: str or list (array-like)
            If str: Heuristic to use to find the elimination order.
            If array-like: The elimination order to use.
            If None: A random elimination order is used.
        """
        # Step 1: Deal with the input arguments.
        if isinstance(variables, str):
            raise TypeError("variables must be a list of strings")
        if isinstance(evidence, str):
            raise TypeError("evidence must be a list of strings")

        # Dealing with the case when variables is not provided.
        if not variables:
            all_factors = []
            for factor_li in self.factors.values():
                all_factors.extend(factor_li)
            if joint:
                return factor_product(*set(all_factors))
            else:
                return set(all_factors)

        # Step 2: Prepare data structures to run the algorithm.
        eliminated_variables = set()
        # Get working factors and elimination order
        # tic = time.time()
        if BP:
            working_factors = self._get_working_factors(variables, evidence, BP=BP)
            elimination_order = self._get_elimination_order(
                variables, evidence, elimination_order=elimination_order, show_progress=show_progress
            )
        else:
            working_factors, sub_graph_model = self._get_working_factors(variables, evidence)
            elimination_order = self._get_elimination_order(
                variables, evidence, sub_graph_model, elimination_order, show_progress=show_progress
            )
        # print(f"getting elimination orders takes {time.time()-toc} secs")
        # Step 3: Run variable elimination
        if show_progress:
            pbar = tqdm(elimination_order)
        else:
            pbar = elimination_order
        print(pbar)
        for var in pbar:
            #tic = time.time()
            # print(var)
            if show_progress:
                pbar.set_description("Eliminating: {var}".format(var=var))
            # Removing all the factors containing the variables which are
            # eliminated (as all the factors should be considered only once)
            factors = [
                factor
                for factor, _ in working_factors[var]
                if not set(factor.variables).intersection(eliminated_variables)
            ]
            phi = factor_product(*factors)
            phi = getattr(phi, operation)([var], inplace=False)
            del working_factors[var]
            for variable in phi.variables:
                if variable in working_factors:
                    working_factors[variable].add((phi, var))
            eliminated_variables.add(var)
            # print(f"eliminating {var} takes {time.time()-tic} secs")

        # Step 4: Prepare variables to be returned.
        #tic = time.time()
        final_distribution = set()
        for node in working_factors:
            for factor, origin in working_factors[node]:
                if not set(factor.variables).intersection(eliminated_variables):
                    final_distribution.add((factor, origin))
        final_distribution = [factor for factor, _ in final_distribution]
        # print(final_distribution)
        # print(f"the rest takes {time.time()-tic} secs")
        if joint:
            if isinstance(self.model, BayesianModel):
                return factor_product(*final_distribution).normalize(inplace=False)
            else:
                return factor_product(*final_distribution)
        else:
            query_var_factor = {}
            for query_var in variables:
                phi = factor_product(*final_distribution)
                query_var_factor[query_var] = phi.marginalize(
                    list(set(variables) - set([query_var])), inplace=False
                ).normalize(inplace=False)
            return query_var_factor

    def query(
            self,
            variables,
            evidence=None,
            elimination_order="weightedminfill",
            joint=True,
            show_progress=False,
            BP=False
    ):
        """
        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: list
            order of variable eliminations (if nothing is provided) order is
            computed automatically

        joint: boolean (default: True)
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`.
        """
        common_vars = set(evidence if evidence is not None else []).intersection(
            set(variables)
        )
        if common_vars:
            raise ValueError(
                f"Can't have the same variables in both `variables` and `evidence`. Found in both: {common_vars}"
            )

        return self._variable_elimination(
            variables=variables,
            operation="marginalize",
            evidence=evidence,
            elimination_order=elimination_order,
            joint=joint,
            show_progress=show_progress,
            BP=BP
        )

In [31]:
class BeliefPropagation(Inference):
    """
    Class for performing inference using Belief Propagation method.

    Creates a Junction Tree or Clique Tree (JunctionTree class) for the input
    probabilistic graphical model and performs calibration of the junction tree
    so formed using belief propagation.

    Parameters
    ----------
    model: BayesianModel, MarkovModel, FactorGraph, JunctionTree
        model for which inference is to performed
    """

    def __init__(self, model):
        super(BeliefPropagation, self).__init__(model)

        if not isinstance(model, JunctionTree):
            self.junction_tree = model.to_junction_tree()
        else:
            self.junction_tree = copy.deepcopy(model)

        self.clique_beliefs = {}
        self.sepset_beliefs = {}

    def get_cliques(self):
        """
        Returns cliques used for belief propagation.
        """
        return self.junction_tree.nodes()

    def get_clique_beliefs(self):
        """
        Returns clique beliefs. Should be called after the clique tree (or
        junction tree) is calibrated.
        """
        return self.clique_beliefs

    def get_sepset_beliefs(self):
        """
        Returns sepset beliefs. Should be called after clique tree (or junction
        tree) is calibrated.
        """
        return self.sepset_beliefs

    def _update_beliefs(self, sending_clique, recieving_clique, operation):
        """
        This is belief-update method.

        Parameters
        ----------
        sending_clique: node (as the operation is on junction tree, node should be a tuple)
            Node sending the message
        recieving_clique: node (as the operation is on junction tree, node should be a tuple)
            Node receiving the message
        operation: str ('marginalize' | 'maximize')
            The operation to do for passing messages between nodes.

        Takes belief of one clique and uses it to update the belief of the
        neighboring ones.
        """
        sepset = frozenset(sending_clique).intersection(frozenset(recieving_clique))
        sepset_key = frozenset((sending_clique, recieving_clique))

        # \sigma_{i \rightarrow j} = \sum_{C_i - S_{i, j}} \beta_i
        # marginalize the clique over the sepset
        sigma = getattr(self.clique_beliefs[sending_clique], operation)(
            list(frozenset(sending_clique) - sepset), inplace=False
        )

        # \beta_j = \beta_j * \frac{\sigma_{i \rightarrow j}}{\mu_{i, j}}
        self.clique_beliefs[recieving_clique] *= (
            sigma / self.sepset_beliefs[sepset_key]
            if self.sepset_beliefs[sepset_key]
            else sigma
        )

        # \mu_{i, j} = \sigma_{i \rightarrow j}
        self.sepset_beliefs[sepset_key] = sigma

    def _is_converged(self, operation):
        """
        Checks whether the calibration has converged or not. At convergence
        the sepset belief would be precisely the sepset marginal.

        Parameters
        ----------
        operation: str ('marginalize' | 'maximize')
            The operation to do for passing messages between nodes.
            if operation == marginalize, it checks whether the junction tree is calibrated or not
            else if operation == maximize, it checks whether the junction tree is max calibrated or not

        Formally, at convergence or at calibration this condition would be satisfied for

        .. math:: \sum_{C_i - S_{i, j}} \beta_i = \sum_{C_j - S_{i, j}} \beta_j = \mu_{i, j}

        and at max calibration this condition would be satisfied

        .. math:: \max_{C_i - S_{i, j}} \beta_i = \max_{C_j - S_{i, j}} \beta_j = \mu_{i, j}
        """
        # If no clique belief, then the clique tree is not calibrated
        if not self.clique_beliefs:
            return False

        for edge in self.junction_tree.edges():
            sepset = frozenset(edge[0]).intersection(frozenset(edge[1]))
            sepset_key = frozenset(edge)
            if (
                edge[0] not in self.clique_beliefs
                or edge[1] not in self.clique_beliefs
                or sepset_key not in self.sepset_beliefs
            ):
                return False

            marginal_1 = getattr(self.clique_beliefs[edge[0]], operation)(
                list(frozenset(edge[0]) - sepset), inplace=False
            )
            marginal_2 = getattr(self.clique_beliefs[edge[1]], operation)(
                list(frozenset(edge[1]) - sepset), inplace=False
            )
            if (
                marginal_1 != marginal_2
                or marginal_1 != self.sepset_beliefs[sepset_key]
            ):
                return False
        return True

    def _calibrate_junction_tree(self, operation):
        """
        Generalized calibration of junction tree or clique using belief propagation. This method can be used for both
        calibrating as well as max-calibrating.
        Uses Lauritzen-Spiegelhalter algorithm or belief-update message passing.

        Parameters
        ----------
        operation: str ('marginalize' | 'maximize')
            The operation to do for passing messages between nodes.

        Reference
        ---------
        Algorithm 10.3 Calibration using belief propagation in clique tree
        Probabilistic Graphical Models: Principles and Techniques
        Daphne Koller and Nir Friedman.
        """
        # Initialize clique beliefs as well as sepset beliefs
        self.clique_beliefs = {
            clique: self.junction_tree.get_factors(clique)
            for clique in self.junction_tree.nodes()
        }
        self.sepset_beliefs = {
            frozenset(edge): None for edge in self.junction_tree.edges()
        }

        for clique in self.junction_tree.nodes():
            if not self._is_converged(operation=operation):
                neighbors = self.junction_tree.neighbors(clique)
                # update root's belief using nieighbor clique's beliefs
                # upward pass
                for neighbor_clique in neighbors:
                    self._update_beliefs(neighbor_clique, clique, operation=operation)
                bfs_edges = nx.algorithms.breadth_first_search.bfs_edges(
                    self.junction_tree, clique
                )
                # update the beliefs of all the nodes starting from the root to leaves using root's belief
                # downward pass
                for edge in bfs_edges:
                    self._update_beliefs(edge[0], edge[1], operation=operation)
            else:
                break

    def calibrate(self):
        """
        Calibration using belief propagation in junction tree or clique tree.
        """
        self._calibrate_junction_tree(operation="marginalize")

    def max_calibrate(self):
        """
        Max-calibration of the junction tree using belief propagation.
        """
        self._calibrate_junction_tree(operation="maximize")

    def _query(
        self, variables, operation, evidence=None, joint=True, show_progress=False
    ):
        """
        This is a generalized query method that can be used for both query and map query.

        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability
        operation: str ('marginalize' | 'maximize')
            The operation to do for passing messages between nodes.
        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence
        References
        ----------
        Algorithm 10.4 Out-of-clique inference in clique tree
        Probabilistic Graphical Models: Principles and Techniques Daphne Koller and Nir Friedman.
        """

        is_calibrated = self._is_converged(operation=operation)
        # Calibrate the junction tree if not calibrated
        if not is_calibrated:
            self.calibrate()

        if not isinstance(variables, (list, tuple, set)):
            query_variables = [variables]
        else:
            query_variables = list(variables)
        query_variables.extend(evidence.keys() if evidence else [])

        # Find a tree T' such that query_variables are a subset of scope(T')
        nodes_with_query_variables = set()
        for var in query_variables:
            nodes_with_query_variables.update(
                filter(lambda x: var in x, self.junction_tree.nodes())
            )
        subtree_nodes = nodes_with_query_variables

        # Conversion of set to tuple just for indexing
        nodes_with_query_variables = tuple(nodes_with_query_variables)
        # As junction tree is a tree, that means that there would be only path between any two nodes in the tree
        # thus we can just take the path between any two nodes; no matter there order is
        for i in range(len(nodes_with_query_variables) - 1):
            subtree_nodes.update(
                nx.shortest_path(
                    self.junction_tree,
                    nodes_with_query_variables[i],
                    nodes_with_query_variables[i + 1],
                )
            )
        subtree_undirected_graph = self.junction_tree.subgraph(subtree_nodes)
        # Converting subtree into a junction tree
        if len(subtree_nodes) == 1:
            subtree = JunctionTree()
            subtree.add_node(subtree_nodes.pop())
        else:
            subtree = JunctionTree(subtree_undirected_graph.edges())

        # Selecting a node is root node. Root node would be having only one neighbor
        if len(subtree.nodes()) == 1:
            root_node = list(subtree.nodes())[0]
        else:
            root_node = tuple(
                filter(lambda x: len(list(subtree.neighbors(x))) == 1, subtree.nodes())
            )[0]
        clique_potential_list = [self.clique_beliefs[root_node]]

        # For other nodes in the subtree compute the clique potentials as follows
        # As all the nodes are nothing but tuples so simple set(root_node) won't work at it would update the set with'
        # all the elements of the tuple; instead use set([root_node]) as it would include only the tuple not the
        # internal elements within it.
        parent_nodes = set([root_node])
        nodes_traversed = set()
        while parent_nodes:
            parent_node = parent_nodes.pop()
            for child_node in set(subtree.neighbors(parent_node)) - nodes_traversed:
                clique_potential_list.append(
                    self.clique_beliefs[child_node]
                    / self.sepset_beliefs[frozenset([parent_node, child_node])]
                )
                parent_nodes.update([child_node])
            nodes_traversed.update([parent_node])

        # Add factors to the corresponding junction tree
        subtree.add_factors(*clique_potential_list)

        # Sum product variable elimination on the subtree
        print(f"nodes: {subtree.nodes}")
        print(f"edges: {subtree.edges}")
        variable_elimination = VariableElimination(subtree, root=False)
        if operation == "marginalize":
            return variable_elimination.query(
                variables=variables,
                evidence=evidence,
                joint=joint,
                show_progress=show_progress,
                BP=True
            )
        elif operation == "maximize":
            return variable_elimination.map_query(
                variables=variables, evidence=evidence, show_progress=show_progress
            )

    def query(self, variables, evidence=None, joint=True, show_progress=False):
        """
        Query method using belief propagation.

        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        joint: boolean
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`.
        """
        common_vars = set(evidence if evidence is not None else []).intersection(
            set(variables)
        )
        if common_vars:
            raise ValueError(
                f"Can't have the same variables in both `variables` and `evidence`. Found in both: {common_vars}"
            )

        result = self._query(
            variables=variables,
            operation="marginalize",
            evidence=evidence,
            joint=joint,
            show_progress=show_progress,
        )
        if joint:
            return result.normalize(inplace=False)
        else:
            return result

In [33]:
BN.model.edges

OutEdgeView([('cat_attr4', 'cat_attr1'), ('cat_attr4', 'cat_attr6'), ('cat_attr1', 'cat_attr3'), ('cat_attr7', 'cat_attr2'), ('cat_attr2', 'cat_attr3'), ('cat_attr5', 'cat_attr6')])

In [34]:
ve = VariableElimination(BN.model)
BN.infer_algo = "exact"
BN.infer_machine = ve

In [43]:
def reduce_graph(model, nodes, root):
    """Returns the minimal part of the tree that contains a set of nodes."""
    sub_nodes = set()

    def walk(node, path):
        if len(nodes) == 0:
            return

        if node in nodes:
            sub_nodes.update(path + [node])
            nodes.remove(node)

        for child in model.successors(node):
            print(child)
            walk(child, path + [node])

    walk(root, [])
    sub_graph = model.subgraph(sub_nodes)
    sub_graph.cardinalities = defaultdict(int)
    for node in sub_graph.nodes:
        sub_graph.cardinalities[node] = model.cardinalities[node]
    return sub_graph

In [40]:
ve.root

'cat_attr4'

In [44]:
m = steiner_tree(BN.model, ['cat_attr1', 'cat_attr5'], ve.root)
print(m.nodes)
print(m.edges)

cat_attr1
cat_attr3
cat_attr6
['cat_attr4', 'cat_attr1']
[('cat_attr4', 'cat_attr1')]


In [36]:
f = ve._get_working_factors(['cat_attr1'], {'cat_attr5': [0, 1, 2, 3]})

KeyError: 'cat_attr5'

In [24]:
bp = BeliefPropagation(BN.model)
BN.infer_algo = "BP"
BN.infer_machine = bp

In [35]:
tic = time.time()
a = BN.query({'cat_attr1':[0,1,2,3], 'cat_attr5': [0, 1, 2, 3]})
print(time.time()-tic)

KeyError: 'cat_attr5'

In [26]:
a

9814.0

In [27]:
len(df.query("cat_attr1 in [0,1,2,3] and cat_attr5 in [0, 1, 2, 3]"))

9814

In [ ]:
from numba import jit
import numpy as np
import time

x = np.arange(100).reshape(10, 10)

class exper:
    def __init__(self):
        self.trace = None
        
    @jit
    def go_fast(self, a): # Function is compiled and runs in machine code
        trace = None
        for i in range(a.shape[0]):
            if trace is None:
                trace = 0.0
            trace += np.tanh(a[i, i])
        return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
e = exper()
start = time.time()
e.go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
e.go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

In [ ]:
class exper:
    def __init__(self):
        self.trace = None
        
    def go_fast(self, a): # Function is compiled and runs in machine code
        trace = None
        for i in range(a.shape[0]):
            if trace is None:
                trace = 0.0
            trace += np.tanh(a[i, i])
        return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
e = exper()
start = time.time()
e.go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))